# Introduction to PyTorch Sorting by way of NumPy

This module explores the reasons to abandon "roll your own" sorting algorithms in deferences to using either NumPy or PyTorch function equivalents.

We took a stab at creating a table to compare sort related functions between NumPy and PyTorch:

Hannesy & Patternson:
"A New Golden Age for Computer Architecture"
https://www.doc.ic.ac.uk/~wl/teachlocal/arch/papers/cacm19golden-age.pdf


| Function | Description | NumPy | PyTorch|
| ---| --- | --- | --- |
| sort | Return a sorted copy of an array. | numpy.sort select kind = ‘quicksort’, ‘mergesort’, ‘heapsort’, ‘stable’ | Cannot specify algorithm |
| argsort | Returns the indices that would sort an array. | numpy.argsort | torch.argsort |
| lexsort | Perform an indirect sort on multiple keys. |  numpy.lexsort | torch.argsort |
| partition | Partially sort an array in-place. | numpy.partition | torch.partition |
| argpartition | Returns the indices that would partition an array. | numpy.argpartition | torch.topk |
| msort | Merge sort an array.  |  numpy.msort | N/A |
| sort_complex | Sort a complex array using the real part first. | numpy.sort_complex | N/A |
| searchsorted | Find indices where elements should be inserted to maintain order. |  numpy.searchsorted | N/A |

Here is a recent news story about Intel accelerating NumPy quicksort:

**Intel Publishes Blazing Fast AVX-512 Sorting Library, Numpy Switching To It For 10~17x Faster Sorts**
- https://www.phoronix.com/news/Intel-AVX-512-Quicksort-Numpy
- Written by Michael Larabel in Intel on 15 February 2023 at 04:00 PM EST. 51 Comments


We will only be investiagating sort in this module.

In [ ]:
import numpy as np
import torch
import time
BIG = 10_000_000
np.random.seed(seed=12)
arr = np.random.rand(BIG)
orig = arr.copy()

In [ ]:
def heapify(arr, n, i):
    largest = i 
    l = 2 * i + 1 
    r = 2 * i + 2 
 
    if l < n and arr[largest] < arr[l]:
        largest = l
 
    if r < n and arr[largest] < arr[r]:
        largest = r
 
    if largest != i:
        arr[i],arr[largest] = arr[largest],arr[i]  
        heapify(arr, n, largest)
    return

def heapSort(arr):
    n = len(arr)
 
    for i in range(n // 2 - 1, -1, -1):
        heapify(arr, n, i)
 
    for i in range(n-1, 0, -1):
        arr[i], arr[0] = arr[0], arr[i] 
        heapify(arr, i, 0)
    return arr

# Heapsort

### The loopy way

In [ ]:
np.random.seed(seed=12)
arr = np.random.rand(BIG)
timing = {}
t1 = time.time()
heapSort(arr)
t2 = time.time()
print("Sorted array is:")
print(arr[:10] )
timing['heapsort_bruteForce'] = time.time() - t1
print('Elapsed time', timing['heapsort_bruteForce'])

### The NumPy way

In [ ]:
np.random.seed(seed=12)
arr = np.random.rand(BIG)
t1 = time.time()
np.sort(arr, axis=None, kind='heapsort') 
t2 = time.time()
print("Sorted array is:")
print(arr[:10] )
timing['heapsort_numpy'] = time.time() - t1
print('Elapsed time', timing['heapsort_numpy'])
print('Numpy Acceleration: {:4.1f} X faster'.format(timing['heapsort_bruteForce']/timing['heapsort_numpy']))

### The PyTorch way

In [ ]:
np.random.seed(seed=12)
arr = np.random.rand(BIG)
arr = torch.tensor(arr)
t1 = time.time()
torch.sort(arr) 
t2 = time.time()
print("Sorted array is:")
print(arr[:10] )
timing['heapsort_pytorch'] = time.time() - t1
print('Elapsed time', timing['heapsort_pytorch'])
print('Numpy Acceleration: {:4.1f} X faster'.format(timing['heapsort_bruteForce']/timing['heapsort_pytorch']))

### Plot the times

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
plt.title("Measure acceleration of looping versus PyTorch/NumPy [Lower is better]",fontsize=12)
plt.ylabel("Time in seconds",fontsize=12)
plt.yscale('log')
plt.xlabel("Various types of operations",fontsize=14)
plt.grid(True)
plt.bar(x = list(timing.keys()), height= list(timing.values()), align='center',tick_label=list(timing.keys()))

# Quicksort

### The loopy way

In [ ]:
def quickSort(arr, low, high):
    if low < high:
        pivotIndex = partition(arr, low, high)
        quickSort(arr, low, pivotIndex - 1)
        quickSort(arr, pivotIndex + 1, high)

def partition(arr, low, high):
    pivot = arr[high]
    i = low - 1  # Index of smaller element
    for j in range(low, high):
        # If current element is smaller than or equal to pivot
        if arr[j] <= pivot:
            i += 1
            arr[i], arr[j] = arr[j], arr[i]
    arr[i+1], arr[high] = arr[high], arr[i+1]
    return i + 1


### The Loopy way

In [ ]:
import time
np.random.seed(seed=12)
arr = np.random.rand(BIG)
timing = {}
t1 = time.time()
quickSort(arr, 0, len(arr)-1)
t2 = time.time()
print("Sorted array is:")
timing['quicksort_bruteForce'] = time.time() - t1
print('Elapsed time', timing['quicksort_bruteForce'])

### The NumPy way

In [ ]:
np.random.seed(seed=12)
arr = np.random.rand(BIG)
t1 = time.time()
np.sort(arr, axis=None, kind='quicksort') 
t2 = time.time()
print("Sorted array is:")
timing['quicksort_numpy'] = time.time() - t1
print('Elapsed time', timing['quicksort_numpy'])
print('Numpy Acceleration: {:4.1f} X faster'.format(timing['quicksort_bruteForce']/timing['quicksort_numpy']))

### The PyTorch way

In [ ]:
np.random.seed(seed=12)
arr = np.random.rand(BIG)
arr = torch.tensor(arr)
t1 = time.time()
torch.sort(arr) 
t2 = time.time()
print("Sorted array is:")
print(arr[:10] )
timing['sort_pytorch'] = time.time() - t1
print('Elapsed time', timing['sort_pytorch'])
print('Numpy Acceleration: {:4.1f} X faster'.format(timing['quicksort_bruteForce']/timing['sort_pytorch']))

### Plot the times

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
plt.title("Measure acceleration of looping versus PyTorch/NumPy [Lower is better]",fontsize=12)
plt.ylabel("Time in seconds",fontsize=12)
plt.yscale('log')
plt.xlabel("Various types of operations",fontsize=14)
plt.grid(True)
plt.bar(x = list(timing.keys()), height= list(timing.values()), align='center',tick_label=list(timing.keys()))

# Mergesort

### The Loopy way

In [ ]:
def mergeSort(arr):
    if len(arr) > 1:
        mid = len(arr) // 2
        leftHalf = arr[:mid]
        rightHalf = arr[mid:]
        
        mergeSort(leftHalf)
        mergeSort(rightHalf)
        
        i = j = k = 0
        
        while i < len(leftHalf) and j < len(rightHalf):
            if leftHalf[i] < rightHalf[j]:
                arr[k] = leftHalf[i]
                i += 1
            else:
                arr[k] = rightHalf[j]
                j += 1
            k += 1
        
        while i < len(leftHalf):
            arr[k] = leftHalf[i]
            i += 1
            k += 1
        
        while j < len(rightHalf):
            arr[k] = rightHalf[j]
            j += 1
            k += 1

### The Loopy way

In [ ]:
import time
np.random.seed(seed=12)
arr = np.random.rand(BIG)
timing = {}
t1 = time.time()
mergeSort(arr)
t2 = time.time()
print("Sorted array is:")
timing['mergesort_bruteForce'] = time.time() - t1
print('Elapsed time', timing['mergesort_bruteForce'])

### The NumPy way

In [ ]:
np.random.seed(seed=12)
arr = np.random.rand(BIG)
t1 = time.time()
np.sort(arr, axis=None, kind='mergesort') 
t2 = time.time()
print("Sorted array is:")
timing['mergesort_numpy'] = time.time() - t1
print('Elapsed time', timing['mergesort_numpy'])
print('Numpy Acceleration: {:4.1f} X faster'.format(timing['mergesort_bruteForce']/timing['mergesort_numpy']))

### The PyTorch way

In [ ]:
np.random.seed(seed=12)
arr = np.random.rand(BIG)
arr = torch.tensor(arr)
t1 = time.time()
torch.sort(arr) 
t2 = time.time()
print("Sorted array is:")
print(arr[:10] )
timing['sort_pytorch'] = time.time() - t1
print('Elapsed time', timing['sort_pytorch'])
print('Numpy Acceleration: {:4.1f} X faster'.format(timing['mergesort_bruteForce']/timing['sort_pytorch']))

### Plot the times

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
plt.title("Measure acceleration of looping versus PyTorch/NumPy [Lower is better]",fontsize=12)
plt.ylabel("Time in seconds",fontsize=12)
plt.yscale('log')

plt.xlabel("Various types of operations",fontsize=14)
plt.grid(True)
plt.bar(x = list(timing.keys()), height= list(timing.values()), align='center',tick_label=list(timing.keys()))

# Notices and Disclaimers

Intel technologies may require enabled hardware, software or service activation.
No product or component can be absolutely secure. 

Your costs and results may vary. 

© Intel Corporation. Intel, the Intel logo, and other Intel marks are trademarks of Intel Corporation or its subsidiaries. Other names and brands may be claimed as the property of others. 